In [2]:
import numpy as np
from PIL import Image
import filepath

from panda3d.core import Texture, CardMaker, NodePath
from panda3d.core import PNMImage, TextureStage

In [6]:
help(tex.setRamImage)

Help on built-in function setRamImage:

setRamImage(...) method of panda3d.core.Texture instance
    C++ Interface:
    set_ram_image(const Texture self, object image, int compression, int page_size)
    
    /**
     * Replaces the current system-RAM image with the new data.  If compression is
     * not CM_off, it indicates that the new data is already pre-compressed in the
     * indicated format.
     *
     * This does *not* affect keep_ram_image.
     */



In [8]:
from util.texture import create_grey_checkerboard

In [15]:
square_size = 512/8
size = (512, 512)  # 图像尺寸 (宽度, 高度)
h = size[0]
w = size[1] # FIXME
# num_squares = 8    # 每行和每列的方格数量
checkerboard_arr = create_grey_checkerboard(size, square_size)

# # 创建棋盘格图像
# # checkerboard_image = create_checkerboard_(size, num_squares)
# checkerboard_image = create_checkerboard(size, square_size)
# # 保存图像
# checkerboard_image.save('checkerboard.png')

# # 显示图像（可选）
# checkerboard_image.show()

In [16]:
tex = Texture()
tex.setup2dTexture(h, w, Texture.T_unsigned_byte, Texture.F_luminance)

In [17]:
# FIXME: transpose or not
# buf = frame[:,:,0].T.tostring()
buf = checkerboard_arr.tobytes()

In [18]:
tex.setRamImage(buf)

In [3]:

# from panda3d.core import LoadPrcFileData, ShowBase

In [19]:
# from panda3d.core import PointLight, DirectionalLight
import filepath
from util.app import ContextShowBase
from util.log import Loggable
from direct.task import Task
import numpy as np
import torch
from geom.basic import create_cube_node, create_sphere_node,uv_curve_surface, create_colored_cube_node
from panda3d.core import (
    Geom,
    GeomNode,
    GeomTriangles,
    GeomVertexData,
    GeomVertexFormat,
    GeomVertexWriter,
    GeomEnums,
    NodePath,
    PointLight,
    DirectionalLight,
    CardMaker,
    WindowProperties
)
import ursina
from direct.showbase import DirectObject
from game.game_object import GameObject
from game.controller import PlayerController
# actions: move forward/backward: by float
# turn left/right
# as functions


# controller
# player controller
# agent controller
from typing import Set, List, Dict,Callable
from datetime import datetime
from abc import ABC
from direct.showbase.ShowBase import ShowBase
from panda3d.core import PNMImage, Texture
from panda3d.core import PTAUchar
from panda3d.core import CardMaker
from panda3d.core import Point2

import numpy as np
# import gizeh as gz
import random

# FIXME: keyboard controller, mouse controller, joystick controller,  etc

class GameSensor:
    pass

# TODO: sensor, logic block, actuator
# TODO: send game env to player interface
# TODO: mouse
    

class CubeController(ABC): # specific for this example
    @classmethod
    def _move_forward(cls, task):
        raise NotImplementedError
    @classmethod
    def _move_backward(cls, task):
        raise NotImplementedError
    @classmethod
    def _turn_left(cls, task):
        raise NotImplementedError
    @classmethod
    def _turn_right(cls, task):
        raise NotImplementedError
    # TODO: check isinstance callable
    def set_move_forward(self, f):
        self._move_forward = f
    
    def set_move_backward(self,f):
        self._move_backward = f
        
    def set_turn_left(self, f):
        self._turn_left = f
        
    def set_turn_right(self, f):
        self._turn_right = f
        
    def call_move_forward(self, task):
        self._move_forward(task)
        
    def call_move_backward(self, task):
        self._move_backward(task)
        
    def call_turn_left(self, task):
        self._turn_left(task)
        
    def call_turn_right(self, task):
        self._turn_right(task)
        
    
class PlayerCubeController(CubeController, PlayerController):
    def __init__(self): # TODO: input keymap dict
        super().__init__()
        # from panda3d.core import KeyboardButton
        self.key_maps = {
            'w': self.call_move_forward,
            's': self.call_move_backward,
            'a': self.call_turn_left,
            'd': self.call_turn_right
        }
            


# class ControlledObject(GameObject):
#      #TODO: bind control
#     def movement(self, task):
#         pass


class ControlledCube(GameObject):
    from panda3d.core import (
        LVector3f,
        LQuaternionf
    )
    def __init__(self, name:str):
        # a cube mesh
        self.cube_node = create_colored_cube_node(
            name, Geom.UHDynamic
        )
        self.geomNodePath = NodePath(self.cube_node)
        self.geomNodePath.setScale(0.2)
        self.scale = 0.2
        self.nodePath = NodePath(name)
        self.geomNodePath.reparent_to(self.nodePath)
        self.move_step = .1
        self.turn_step=5
        
    def register_controller(self, controller: CubeController):
        self.controller = controller
        controller.set_move_forward(
            lambda task: self.move(task, dist=self.move_step)
        )
        controller.set_move_backward(
            lambda task: self.move(task, dist=-self.move_step)
        )
        controller.set_turn_left(
            lambda task: self.rotate(task, angle=self.turn_step)
        )
        controller.set_turn_right(
            lambda task: self.rotate(task, angle=-self.turn_step)
        )
        
    def setPos(self,*pos):
        self.nodePath.setPos(*pos)
        
    def setX(self, x):
        self.nodePath.setX(x)
        
    def setY(self, y):
        self.nodePath.setY(y)
        
    def setZ(self, z):
        self.nodePath.setZ(z)
        
    # TODO: setHpr
    def move(self, task:Task, dist:float):
        # self.log("move {}".format(dist))
        new_pos = self.getPos() + self.getForward(self.ref) * dist
        self.setPos(new_pos)
        
    def rotate(self, task: Task, angle:float):
        new_H = self.nodePath.getH() + angle
        self.nodePath.setH(new_H)
    
    def setRef(self, ref):
        self.ref = ref
        
    def getPos(self): # FIXME: args
        return self.nodePath.getPos()
        
    def getQuat(self, ref) -> LQuaternionf:
        return self.nodePath.getQuat(ref)
    
    def getForward(self, ref) -> LVector3f:
        return self.nodePath.getQuat(ref).getForward()
class Movement(ContextShowBase):
    def __init__(self):
        super().__init__()
        self.name = "movement"
        image_x_size = 512
        image_y_size = 512

        # Quad in scene, to display the image on
        # input_img = PNMImage(image_x_size, image_y_size)  # It's RGB.
        input_tex = Texture()
        # input_tex.load(input_img)
        card = CardMaker('in_scene_screen')
        card.setFrame(-5,5,-5,5)
        card.setUvRange(Point2(0, 1),  # ll
                        Point2(1, 1),  # lr
                        Point2(1, 0),  # ur
                        Point2(0, 0))  # ul
        sky_card = CardMaker("sky")
        sky_card.setFrame(-20,20,-20,20)
        sky_card.setUvRange(Point2(0, 1),  # ll
                        Point2(1, 1),  # lr
                        Point2(1, 0),  # ur
                        Point2(0, 0))
        
        screen = render.attach_new_node(card.generate())
        screen.set_pos(0,0,0)
        screen.set_texture(input_tex)
        self.screen = screen
        screen.setHpr(0,-90,0)
        sky = render.attach_new_node(sky_card.generate())
        sky.set_pos(5,5,10)
        sky.setColor((0.87,1,1,1))
        sky.setHpr(0,90,0)
        self.camera.setPos(5,5,10)
        self.accept('z', self.toggle_camera)
        # self.camera.setHpr(0,-90,0)
        
        
        sphere_o = create_sphere_node("o", 12, 12)
        self.render.attachNewNode(sphere_o)
        node_path_sphere_o = NodePath(sphere_o)
        node_path_sphere_o.setColor((1,1,1,1))
        node_path_sphere_o.setPos(0,0,0)
        sphere_x = create_sphere_node("x", 12, 12)
        self.render.attachNewNode(sphere_x)
        node_path_sphere_x = NodePath(sphere_x)
        node_path_sphere_x.setColor((1,0,0,1))
        node_path_sphere_x.setPos(1,0,0)
        sphere_y = create_sphere_node("y", 12, 12)
        self.render.attachNewNode(sphere_y)
        node_path_sphere_y = NodePath(sphere_y)
        node_path_sphere_y.setColor((0,1,0,1))
        node_path_sphere_y.setPos(0,1,0)
        sphere_z = create_sphere_node("z", 12, 12)
        self.render.attachNewNode(sphere_z)
        node_path_sphere_z = NodePath(sphere_z)
        node_path_sphere_z.setColor((0,0,1,1))
        node_path_sphere_z.setPos(0,0,1)
        node_path_sphere_o.setScale(0.2)
        node_path_sphere_x.setScale(0.2)
        node_path_sphere_y.setScale(0.2)
        node_path_sphere_z.setScale(0.2)
        
        # create a cube
        self.player_cube =  ControlledCube("player")
        self.player_cube.reparentTo(self.render)
        self.player_cube.setZ(self.player_cube.scale/2)
        self.player_cube.setRef(self.render)
        self.controller = PlayerCubeController()
        self.player_cube.register_controller(self.controller)
        self.buttonThrowers[0].node().setButtonDownEvent('button')
        self.buttonThrowers[0].node().setButtonUpEvent('button-up')
        # print("orient", self.player_cube.nodePath.getQuat(render))
        # orient 1 + 0i + 0j + 0k
        # print("orient", self.player_cube.nodePath.getQuat(render).getForward())
        # orient LVector3f(0, 1, 0)
     
    # 隐藏鼠标光标
        props = WindowProperties()
        props.setCursorHidden(True)
        self.win.requestProperties(props)

        # 将鼠标居中
        self.center_mouse()
        # 禁用默认的鼠标控制（这样摄像机就不会被默认控制）
        self.disableMouse()

        # 设定初始摄像机位置
        self.camera.setPos(0, -10, 1)

        # 每帧更新摄像机
        self.taskMgr.add(self.update_camera, "update_camera_task")
        # self.taskMgr.add(self.spin_card, "spin_card_task")
        self.taskMgr.add(self.controller.update, "update_controller")

        # 保存鼠标的初始位置
        self.prev_mouse_x = 0
        self.prev_mouse_y = 0

    def center_mouse(self):
        """将鼠标指针重置到窗口的中心"""
        window_center_x = self.win.getXSize() // 2
        window_center_y = self.win.getYSize() // 2
        self.win.movePointer(0, window_center_x, window_center_y)
        self.prev_mouse_x = window_center_x
        self.prev_mouse_y = window_center_y
        
    def toggle_camera(self):
        self.camera.setHpr(0,0,0)

    def update_camera(self, task):
        """每帧更新摄像机的方向，使其跟随鼠标的移动"""
        if self.mouseWatcherNode.hasMouse():
            # 获取鼠标的位置（归一化的 -1 到 1 范围内）
            mouse_x = self.win.getPointer(0).getX()
            mouse_y = self.win.getPointer(0).getY()

            # 计算鼠标移动的增量
            delta_x = (mouse_x - self.prev_mouse_x)
            delta_y = mouse_y - self.prev_mouse_y

            # 调整摄像机的水平旋转和俯仰角度
            camera_h = self.camera.getH() - delta_x * 0.1
            camera_p = self.camera.getP() - delta_y * 0.1

            # 设置新的摄像机角度
            self.camera.setH(camera_h)
            self.camera.setP(camera_p)

            # 将鼠标指针重置到窗口的中心
            self.center_mouse()

        return task.cont
    
    def spin_card(self, task):
        time_ = task.time
        angleDegrees = time_ * 60.0

        # self.screen.setH(angleDegrees) # around Z
        # self.screen.setP(angleDegrees)  # around X
        return Task.cont
        

#     def spin_camera_task(self, task):
#         # time_ = task.time
#         time_=60
#         angleDegrees = time_ * 60.0
#         angleRadians = angleDegrees * (np.pi / 180.0)
#         self.camera.setPos(20 * np.sin(angleRadians), -20 * np.cos(angleRadians), 3)
#         self.camera.setHpr(angleDegrees, 0, 0)
#         # self.camera.setPos(1,1,3)
#         # self.camera.setHpr(90,0,0)

#         return Task.cont

    def toggle_fullscreen(self):
        props = WindowProperties()
        props.setFullscreen(not self.win.isFullscreen())
        self.win.requestProperties(props)
        
    # TODO: pause game
    # TODO: resume game

# Gizeh setup
# surface = gz.Surface(image_x_size, image_y_size)


# def update_gizeh_image():
#     star1 = gz.star(radius=70, ratio=.4, fill=(1,1,1), angle=-np.pi/2,
#                     stroke_width=2, stroke=(1,0,0))
#     star2 = gz.star(radius=55, ratio=.4, fill=(1,0,0), angle=-np.pi/2)
#     # Gizeh coords are right-down.
#     stars = gz.Group([star1, star2]).translate([random.randint(100,412),
#                                                 random.randint(100,412)])
#     stars.draw(surface)


# def gizeh_image(task):
#     # Why does this need to be copied? That might be expensive!
#     #   Some manipulations apparently make an array non-viable for transfer, and
#     #   that includes even an out-of-the-box gizeh surface.
#     input_tex.set_ram_image_as(PTAUchar(surface.get_npimage().copy()),
#                                'RGB')
#     return task.cont


# base.taskMgr.add(gizeh_image, "gizeh to screen", sort=45)

# s.run()
import builtins
import traceback
try:
    with Movement() as app:
        app.run()
except Exception as e:
    print(e)
    print(traceback.format_exc())
finally:
    if hasattr(builtins, 'base'):
        builtins.base.destroy()

Known pipe types:
  glxGraphicsPipe
(all display modules loaded.)
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager
:audio(error): Couldn't open default OpenAL device
:audio(error): OpenALAudioManager: No open device or context
:audio(error):   OpenALAudioManager is not valid, will use NullAudioManager


---movement run(), at 2024-08-25 10:44:59.104293---
* interrupt by keyboard
---movement destroy at 2024-08-25 10:45:05.111605---
---movement destroyed at 2024-08-25 10:45:05.128752, exit---


In [4]:
size = (512,512)
square_size = 512/8
cb = create_grey_checkerboard(size, square_size)
cbpnm = np2pnm(cb)


TypeError: set_xel_val() takes 4 or 6 arguments (2 given)

In [27]:
t = Texture()

In [29]:
t.load(checkerboard_image)

TypeError: Arguments must match:
load(const Texture self, const PNMImage pnmimage)
load(const Texture self, const PfmFile pfm)
load(const Texture self, const PNMImage pnmimage, const LoaderOptions options)
load(const Texture self, const PfmFile pfm, const LoaderOptions options)
load(const Texture self, const PNMImage pnmimage, int z, int n, const LoaderOptions options)
load(const Texture self, const PfmFile pfm, int z, int n, const LoaderOptions options)
